In [34]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from datastack.ml import bestmatch
reload(bestmatch)
from datastack.ml import bestmatch as bm
from sklearn.linear_model import LogisticRegression
import rosetta_settings
reload(rosetta_settings)
import rosetta_settings as ml_vars

In [20]:
df_obs = pd.read_table("s3://hli-mv-data-science/cmaher/SelectData/paper.generator.real.csv", sep=",", index_col=0)

In [21]:
df_pred = pd.read_table("s3://hli-mv-data-science/cmaher/SelectData/paper.generator.predictions.csv", sep=",", index_col=0)
df_pred.columns = ["%s_p" % xx for xx in df_pred.columns]

In [22]:
full_df = pd.concat([df_obs, df_pred], axis=1)

Index([u'pheno.age', u'pheno.age_p'], dtype='object')

In [23]:
eth_cols =  df_obs.filter(like="ethnicity").columns

Index([u'dynamic.FACE.ethnicity.region5.v1.afr',
       u'dynamic.FACE.ethnicity.region5.v1.amr',
       u'dynamic.FACE.ethnicity.region5.v1.csa',
       u'dynamic.FACE.ethnicity.region5.v1.eas',
       u'dynamic.FACE.ethnicity.region5.v1.eur'],
      dtype='object')
(792, 250)


In [25]:
from datastack.dbs.rdb import RosettaDBMongo
db = RosettaDBMongo(host='10.6.36.96')
db.initialize(namespace='hg38_noEBV', version='1447895375')
keys = db.find_keys('(dynamic.FACE.related.v1.value)|(dynamic.FACE.consented.v1.value)|(%s)|(%s)|(%s)' % (ml_vars.SAMPLE_NAME,
                                                                                                     ml_vars.PROJECT_KEY,
                                                                                                         ml_vars.SAMPLE_KEY), 
                    regex=True)
df = db.query(keys, filters={"lab.ProjectID":"FACE"})

In [26]:
fulldf_joined = df.set_index(ml_vars.SAMPLE_KEY).join(full_df)

In [27]:
fulldf_joined.dropna().head()

,ds.index.ProjectID,ds.index.sample_name,dynamic.FACE.consented.v1.value,dynamic.FACE.related.v1.value,pheno.gender,pheno.age,dynamic.FACE.ethnicity.region5.v1.afr,dynamic.FACE.ethnicity.region5.v1.amr,dynamic.FACE.ethnicity.region5.v1.csa,dynamic.FACE.ethnicity.region5.v1.eas,...,dynamic.FACE.2dmouth.v2.visit1.PC.1_p,dynamic.FACE.2dmouth.v2.visit1.PC.2_p,dynamic.FACE.2dmouth.v2.visit1.PC.3_p,dynamic.FACE.2dmouth.v2.visit1.PC.4_p,dynamic.FACE.2dmouth.v2.visit1.PC.5_p,dynamic.FACE.2dface.v1.visit1.PC.1_p,dynamic.FACE.2dface.v1.visit1.PC.2_p,dynamic.FACE.2dface.v1.visit1.PC.3_p,dynamic.FACE.2dface.v1.visit1.PC.4_p,dynamic.FACE.2dface.v1.visit1.PC.5_p
ds.index.sample_key,,,,,,,,,,,,,,,,,,,,,
88535_187525496,FACE,187525496,False,[],-1,22,0.547066,0.289240,0.003071,0.089649,...,-409.993425,2233.411207,269.357715,-15.068942,122.059770,2609.820083,26.103015,59.854380,112.719093,-144.109573
78514_187522203,FACE,187522203,False,"[""187522230""]",-1,39,0.871085,0.011677,0.000000,0.011047,...,146.889783,1391.260447,346.694088,-26.015962,85.307360,1534.752643,197.399290,41.786995,-418.399679,-37.431558
85103_176444178,FACE,176444178,False,[],-1,65,0.651376,0.000000,0.000000,0.000000,...,1444.828620,293.242170,23.077919,-26.717473,-16.956094,-676.027364,888.712420,-18.542032,-542.588987,661.705997
78503_187522202,FACE,187522202,False,[],-1,24,0.782863,0.000000,0.000000,0.000000,...,-642.736458,453.274430,278.034529,-16.370351,-13.435942,1368.366766,503.715359,-18.673643,-25.312388,131.235500
86005_187526055,FACE,187526055,False,[],-1,33,0.000000,0.000000,0.008389,0.991581,...,1698.392374,733.206241,-475.719456,46.076866,-5.613614,-1661.473136,-578.698922,194.163476,-183.793892,-379.599402


In [28]:
fulldf_joined.notnull().all(axis=1).sum()

792

In [37]:
obs_cols = list(eth_cols) + ['pheno.age']
pred_cols = ["%s_p" % xx for xx in obs_cols]

In [38]:
es = bm.EvalSelect(fulldf_joined.reset_index().copy(), bm.WeightedNearestNeighbor(), from_cols=pred_cols, to_cols=obs_cols)
es.fit()
es.predict()

Done fetching lineups


match            0.532
select           0.480
select_unique    0.498
match_query      0.560
select_query     0.480
dtype: float64

In [39]:
col_groups = ["1"] * len(eth_cols) + ["2"]
es2 = bm.EvalSelect(fulldf_joined.reset_index().copy(), bm.WeightedNearestNeighbor(), 
                    from_cols=pred_cols, to_cols=obs_cols,
                    col_groups=col_groups
                   )
es2.fit()
es2.predict()

Done fetching lineups


match            0.561
select           0.472
select_unique    0.507
match_query      0.570
select_query     0.480
dtype: float64